In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import xml.etree.ElementTree as ET
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def parse_relative_time(relative_time):
    relative_time = relative_time.lower()
    now = datetime.now()

    if "segundos" in relative_time or "minutos" in relative_time:
        # Extrair o número de minutos
        minutes = int(relative_time.split()[1])
        return now - timedelta(minutes=minutes)

    elif "horas" in relative_time:
        # Extrair o número de horas
        hours = int(relative_time.split()[1])
        return now - timedelta(hours=hours)

    elif "ontem" in relative_time:
        # Ontem
        return now - timedelta(days=1)

    elif "dia" in relative_time or "dias" in relative_time:
        # Extrair o número de dias
        days = int(relative_time.split()[1])
        return now - timedelta(days=days)

    elif "semana" in relative_time or "semanas" in relative_time:
        # Extrair o número de semanas
        weeks = int(relative_time.split()[1])
        return now - timedelta(weeks=weeks)

    elif "mês" in relative_time or "meses" in relative_time:
        # Extrair o número de meses
        months = int(relative_time.split()[1])
        # Assume-se que um "mês" tem 30 dias para simplificar
        return now - timedelta(days=months * 30)

    else:
        return None


### Extrair Notícias Brazil Journal

In [3]:
def getNewsBrazilJournal(url, tag):

  # Inicializando o DataFrame com None
  df = None

  # Fazendo a solicitação GET
  response = requests.get(url)

  # Verificando se a solicitação foi bem-sucedida
  if response.status_code == 200:
      # Obtendo o conteúdo HTML
      html = response.text

      # Parseando o HTML com BeautifulSoup
      soup = BeautifulSoup(html, "html.parser")

      # Encontrando todas as notícias
      articles = soup.find_all("article", class_="boxarticle")

      # Lista para armazenar os dados de cada notícia
      data = []

      # Iterando sobre cada notícia para extrair título, link e data
      for article in articles:
          # print(article)
          title = article.find("h2", class_="boxarticle-infos-title").text.strip()
          link = article.find("h2", class_="boxarticle-infos-title").find("a")["href"]  # Extrai a URL da notícia
          img_elem = article.find("img")  # Encontra a tag img dentro da tag a
          img_url = img_elem["src"] if img_elem else None  # Extrai a URL da imagem, se existir


          # Fazendo solicitação GET para o link da notícia
          response_article = requests.get(link)

          # Verificando se a solicitação para o link da notícia foi bem-sucedida
          if response_article.status_code == 200:
              # Obtendo o conteúdo HTML da notícia
              html_article = response_article.text

              # Parseando o HTML da notícia com BeautifulSoup
              soup_article = BeautifulSoup(html_article, "html.parser")
              # print(soup_article)

              # Encontrando a data de publicação da notícia, se existir
              post_time_elem = soup_article.find("div", class_="post-header-info")
              post_time = post_time_elem.text.strip() if post_time_elem else None


              # Adicionando os dados da notícia à lista de dados
              data.append({
                  "titulo": title,
                  "url": link,  # Agora a URL da notícia é usada aqui
                  "descricao": "",
                  "data_publicacao": post_time,
                  "img": img_url,
                  "fonte": "Brazil Journal"
              })
          else:
              print("Falha ao acessar o link da notícia:", link)

      # Criando o DataFrame com os dados das notícias
      df = pd.DataFrame(data)

  else:
      print("Falha ao acessar o site. Status code:", response.status_code)

  return df

# URL do site
urlBrazilJournal = [
        {"url": "https://braziljournal.com/tag/petroleo/", "tag": "petroleo"},
        {"url": "https://braziljournal.com/tag/setor-eletrico/", "tag": "setor-eletrico"},
        {"url": "https://braziljournal.com/tag/servicos-financeiros/", "tag": "servicos-financeiros"},
        {"url": "https://braziljournal.com/tag/varejo-e-consumo/", "tag": "varejo-e-consumo"},
        {"url": "https://braziljournal.com/tag/mineracao/", "tag": "mineracao"},
        {"url": "https://braziljournal.com/tag/siderurgia/", "tag": "siderurgia"},
        {"url": "https://braziljournal.com/tag/industria-automotiva/", "tag": "industria-automotiva"},
        {"url": "https://braziljournal.com/tag/agronegocio/", "tag": "agronegocio"},
        {"url": "https://braziljournal.com/tag/arte/", "tag": "arte"},
        {"url": "https://braziljournal.com/tag/beleza-e-cosmeticos/", "tag": "beleza-e-cosmeticos"},
        {"url": "https://braziljournal.com/tag/drogaria/", "tag": "drogaria"}
      ]

# Inicializando o DataFrame
df_bj = pd.DataFrame()

for url in urlBrazilJournal:
  df = getNewsBrazilJournal(url["url"], url["tag"])
  df_bj = pd.concat([df, df_bj])

# dropar linhas None de publicacao
df_bj = df_bj.dropna(subset=['data_publicacao'])

df_bj


KeyboardInterrupt: 

In [4]:
# URL do site
url = "https://braziljournal.com/"

# Fazendo a solicitação GET
response = requests.get(url)

# Verificando se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Obtendo o conteúdo HTML
    html = response.text

    # Parseando o HTML com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Encontrando todas as notícias
    articles = soup.find_all("article", class_="boxarticle")

    # Lista para armazenar os dados de cada notícia
    data = []

    # Iterando sobre cada notícia para extrair título, link e data
    for article in articles:
        title = article.find("h2", class_="boxarticle-infos-title").text.strip()
        link = article.find("a")["href"]
        img_elem = article.find("img")  # Encontra a tag img dentro da tag a
        img_url = img_elem["src"] if img_elem else None  # Extrai a URL da imagem, se existir
        # tag = article.find("p", class_="boxarticle-infos-tag").text.strip()

        # Fazendo solicitação GET para o link da notícia
        response_article = requests.get(link)

        # Verificando se a solicitação para o link da notícia foi bem-sucedida
        if response_article.status_code == 200:
            # Obtendo o conteúdo HTML da notícia
            html_article = response_article.text

            # Parseando o HTML da notícia com BeautifulSoup
            soup_article = BeautifulSoup(html_article, "html.parser")

            # Encontrando a data de publicação da notícia, se existir
            post_time_elem = soup_article.find("div", class_="post-header-info")
            post_time = post_time_elem.text.strip() if post_time_elem else None

            # Adicionando os dados da notícia à lista de dados
            data.append({
                "titulo": title,
                "url": link,
                "descricao": "saiba mais a sobre "+title,
                "data_publicacao": post_time,
                "img": img_url,
                "fonte": "Brazil Journal"
            })
        else:
            print("Falha ao acessar o link da notícia:", link)

    # Criando o DataFrame com os dados das notícias
    df_bj_p = pd.DataFrame(data)

else:
    print("Falha ao acessar o site. Status code:", response.status_code)

df_bj_p = df_bj_p.dropna(subset=['data_publicacao'])
df_bj_p


,titulo,url,descricao,data_publicacao,img,fonte
42,Puig – dona da Carolina Herrera e Jean Paul Ga...,https://braziljournal.com/puig-dona-da-carolin...,saiba mais a sobre Puig – dona da Carolina Her...,8 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
47,A crise dos restaurantes estrelados,https://braziljournal.com/a-crise-dos-restaura...,saiba mais a sobre A crise dos restaurantes es...,7 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
52,"Alife Nino, dono do Nino Cucina e Tatu Bola, c...",https://braziljournal.com/alife-nino-dono-do-n...,"saiba mais a sobre Alife Nino, dono do Nino Cu...",5 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
53,Trump Media: bolha ou startup?,https://braziljournal.com/trump-media-bolha-ou...,saiba mais a sobre Trump Media: bolha ou startup?,5 de abril de 2024,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal


### Extrair Notícias do Valor Econômico

In [5]:
# Lista para armazenar os dados de todas as páginas
all_data = []

# Loop sobre cada página até a página 100
for page_number in range(1, 11):
    # URL da página atual
    url = f"https://valor.globo.com/financas/index/feed/pagina-{page_number}"

    # Fazendo a solicitação GET
    response = requests.get(url)

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Obtendo o conteúdo HTML
        html = response.text

        # Parseando o HTML com BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        # Encontrando todas as notícias
        articles = soup.find_all("div", class_="feed-post-body")

        # Iterando sobre cada notícia para extrair título, link, data e imagem
        for article in articles:
            title_elem = article.find("h2", class_="feed-post-link")
            link_elem = article.find("a")
            description_elem = article.find("p", class_="feed-post-body-resumo")
            img_elem = article.find("img")
            post_time_elem = article.find("span", class_="feed-post-datetime")

            # Definindo valores padrão para elementos ausentes
            title = title_elem.text.strip() if title_elem else "Título não disponível"
            link = link_elem["href"] if link_elem else "Link não disponível"
            description = description_elem.text.strip() if description_elem else "Descrição não disponível"
            img = img_elem["src"] if img_elem else "Imagem não disponível"
            post_time = post_time_elem.text.strip() if post_time_elem else "Data de publicação não disponível"

            # Adicionando os dados da notícia à lista de dados de todas as páginas
            all_data.append({
                "titulo": title,
                "url": link,
                "descricao": description,
                "data_publicacao": post_time,
                "img": img,
                "fonte": "Valor Econômico"
            })

        print(f"Página {page_number} processada.")
    else:
        print(f"Falha ao acessar a página {page_number}. Status code:", response.status_code)

# Criando o DataFrame com os dados de todas as páginas
df_valor = pd.DataFrame(all_data)

df_valor["data_publicacao"] = df_valor["data_publicacao"].apply(parse_relative_time)


Página 1 processada.
Página 2 processada.
Página 3 processada.
Página 4 processada.
Página 5 processada.
Página 6 processada.
Página 7 processada.
Página 8 processada.
Página 9 processada.
Página 10 processada.


### Extrair notícias Infomoney

In [6]:
# Função para ler o XML da URL e retornar um DataFrame
def xml_to_dataframe(url):
    # Fazendo a requisição GET para obter o conteúdo da URL
    response = requests.get(url)

    # Verificando se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Analisando o XML
        root = ET.fromstring(response.content)

        # Inicializando listas para armazenar os dados
        titles = []
        links = []
        descriptions = []
        pub_dates = []
        images = []  # Nova lista para armazenar os URLs das imagens

        # Expressão regular para encontrar a tag <img> e seu conteúdo
        img_regex = re.compile(r'<img[^>]+src="([^"]+)"[^>]*>')

        # Iterando sobre os itens no XML
        for item in root.findall('.//item'):
            title = item.find('title').text
            link = item.find('link').text
            description = item.find('description').text.replace('<br />', '').replace("<p>", "").replace("</p>", "")
            pub_date = item.find('pubDate').text

            # Procurando por tags <img> na descrição e extraindo o URL da imagem
            img_match = img_regex.search(description)
            image_url = img_match.group(1) if img_match else None

            # Removendo a tag <img> da descrição
            description = img_regex.sub('', description)

            # Adicionando os dados às listas
            titles.append(title)
            links.append(link)
            descriptions.append(description)
            pub_dates.append(pub_date)
            images.append(image_url)

        # Criando um dicionário com os dados
        data = {
            'titulo': titles,
            'url': links,
            'descricao': descriptions,
            'data_publicacao': pub_dates,
            'img': images,
            "fonte": "InfoMoney"  # Alterando a fonte para "InfoMoney"
        }

        # Criando o DataFrame
        df = pd.DataFrame(data)
        return df
    else:
        print("Erro ao obter o XML. Código de status:", response.status_code)
        return None

# URL do XML do InfoMoney
url = "https://www.infomoney.com.br/feed/"

# Chamando a função para criar o DataFrame
df_infomoney = xml_to_dataframe(url)

# Exibindo o DataFrame
df_infomoney


,titulo,url,descricao,data_publicacao,img,fonte
0,Lula assina MP que promete reduzir conta de lu...,https://www.infomoney.com.br/politica/lula-ass...,Texto abre possibilidade para governo antecipa...,"Tue, 09 Apr 2024 14:40:42 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
1,"Sabesp (SBSP3) vai reajustar as tarifas em 6,4...",https://www.infomoney.com.br/mercados/sabesp-s...,Reajuste passa a vigorar em maio e irá vigorar...,"Tue, 09 Apr 2024 14:35:38 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
2,Dividendos da Petrobras (PETR4): acordo sobre ...,https://www.infomoney.com.br/mercados/dividend...,"Governo pode remunerar a mais os acionistas, c...","Tue, 09 Apr 2024 14:28:41 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
3,CEO do JPMorgan diz que tendências de longo pr...,https://www.infomoney.com.br/economia/ceo-do-j...,Jamie Dimon afirma na carta anual do banco que...,"Tue, 09 Apr 2024 14:16:26 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
4,Ibovespa Ao Vivo: Bolsa sobe com PETR4 e grand...,https://www.infomoney.com.br/mercados/ibovespa...,Bolsas dos EUA avançam antes de dados de infla...,"Tue, 09 Apr 2024 14:01:50 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
5,Calendário do Bolsa Família 2024: confira as d...,https://www.infomoney.com.br/minhas-financas/c...,"No mês de abril, o benefício estará disponível...","Tue, 09 Apr 2024 14:00:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
6,"Investidores da Tesla, de Musk, se preparam pa...",https://www.infomoney.com.br/mercados/investid...,Companhia encara comparação difícil na base an...,"Tue, 09 Apr 2024 13:57:57 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
7,Champions League retorna hoje e França e Espan...,https://www.infomoney.com.br/mundo/champions-l...,Espanha sedia hoje o jogo entre Real Madrid e ...,"Tue, 09 Apr 2024 13:45:26 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
8,"Funcionários do X foram ameaçados de prisão, d...",https://www.infomoney.com.br/politica/funciona...,"Elon Musk, dono do X (antigo Twitter), respond...","Tue, 09 Apr 2024 13:45:00 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney
9,Elon Musk prevê que IA será mais esperta que o...,https://www.infomoney.com.br/business/elon-mus...,"O bilionário, que também foi cofundador da Ope...","Tue, 09 Apr 2024 13:38:03 +0000",https://www.infomoney.com.br/wp-content/upload...,InfoMoney


### Concatenar as Fontes

In [7]:
df = pd.concat([df_bj, df_bj_p, df_infomoney, df_valor])

def converter_data(valor):
    try:
        # Tenta converter para data diretamente
        return pd.to_datetime(valor)
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X dias atrás'
    if 'dias atrás' or 'dia atrás' in valor:
        dias_atras = int(valor.split()[0])
        return datetime.now() - relativedelta(days=dias_atras)

    # Verifica se o valor está no formato 'X semanas atrás'
    if 'semanas atrás' or 'semana atrás' in valor:
        semanas_atras = int(valor.split()[0])
        return datetime.now() - relativedelta(weeks=semanas_atras)

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%d de %B de %Y')
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%d/%m/%Y %H:%M:%S')
    except ValueError:
        pass

    # Verifica se o valor está no formato 'X de mês de ano'
    try:
        return pd.to_datetime(valor, format='%a, %d %b %Y %H:%M:%S %z')
    except ValueError:
        pass

    # Se não se encaixar em nenhum dos formatos conhecidos, retorna NaN
    return pd.NaT

# Aplicando a função de conversão à coluna 'data'
df['data_publicacao'] = df['data_publicacao'].apply(converter_data)

df

,titulo,url,descricao,data_publicacao,img,fonte
42,Puig – dona da Carolina Herrera e Jean Paul Ga...,https://braziljournal.com/puig-dona-da-carolin...,saiba mais a sobre Puig – dona da Carolina Her...,2024-04-01 14:41:13.437326,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
47,A crise dos restaurantes estrelados,https://braziljournal.com/a-crise-dos-restaura...,saiba mais a sobre A crise dos restaurantes es...,2024-04-02 14:41:13.437849,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
52,"Alife Nino, dono do Nino Cucina e Tatu Bola, c...",https://braziljournal.com/alife-nino-dono-do-n...,"saiba mais a sobre Alife Nino, dono do Nino Cu...",2024-04-04 14:41:13.438280,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
53,Trump Media: bolha ou startup?,https://braziljournal.com/trump-media-bolha-ou...,saiba mais a sobre Trump Media: bolha ou startup?,2024-04-04 14:41:13.438687,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
0,Lula assina MP que promete reduzir conta de lu...,https://www.infomoney.com.br/politica/lula-ass...,Texto abre possibilidade para governo antecipa...,2024-04-09 14:40:42+00:00,https://www.infomoney.com.br/wp-content/upload...,InfoMoney
...,...,...,...,...,...,...
95,Há risco de Fed precisar voltar a elevar juros...,https://valor.globo.com/financas/noticia/2024/...,Michelle Bowman disse que esse não é seu cenár...,2024-04-05 14:40:46.807375,https://s2-valor.glbimg.com/20dVrHtvoFOUhyQbfn...,Valor Econômico
96,Clientes do C6 Bank relatam instabilidade no Pix,https://valor.globo.com/financas/noticia/2024/...,A principal queixa é que transferências realiz...,2024-04-05 14:40:46.807376,https://s2-valor.glbimg.com/ILV8pCrEfd-EynbS04...,Valor Econômico
97,Bolsas da Europa recuam com dados fortes de em...,https://valor.globo.com/financas/noticia/2024/...,"O índice Stoxx 600 caiu 0,88% e anotou seu men...",2024-04-05 14:40:46.807378,https://s2-valor.glbimg.com/7OJTblUtG1oKbqkNyn...,Valor Econômico
98,Fundos de shoppings têm melhor retorno entre F...,https://valor.globo.com/financas/noticia/2024/...,"Segundo o relatório, em 12 meses até março, o ...",2024-04-05 14:40:46.807380,https://s2-valor.glbimg.com/6efJvpNGUZM1ko2Wnm...,Valor Econômico


### Calculo de Similaridade - Verificar se as noticias estão repetindo

In [8]:
# Criando um vetorizador TF-IDF para converter o texto das notícias em vetores numéricos
vectorizer = TfidfVectorizer()

# Convertendo o texto das notícias em vetores TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['titulo'])

# Calculando a similaridade de cosseno entre os vetores TF-IDF
similaridade = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Definindo um limite de similaridade para considerar duas notícias como iguais
limite_similaridade = 0.8

# Lista para armazenar os índices das notícias únicas
indices_noticias_unicas = []

# Procurando por notícias semelhantes
for i in range(len(similaridade)):
    duplicada = False
    for j in range(i+1, len(similaridade)):
        if similaridade[i,j] > limite_similaridade:
            duplicada = True
            break
    if not duplicada:
        indices_noticias_unicas.append(i)

# Selecionando apenas as notícias únicas do dataframe original
df_noticias_unicas = df.iloc[indices_noticias_unicas]

# Agora 'df_noticias_unicas' contém apenas as notícias únicas
df = df_noticias_unicas

df

,titulo,url,descricao,data_publicacao,img,fonte
42,Puig – dona da Carolina Herrera e Jean Paul Ga...,https://braziljournal.com/puig-dona-da-carolin...,saiba mais a sobre Puig – dona da Carolina Her...,2024-04-01 14:41:13.437326,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
47,A crise dos restaurantes estrelados,https://braziljournal.com/a-crise-dos-restaura...,saiba mais a sobre A crise dos restaurantes es...,2024-04-02 14:41:13.437849,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
52,"Alife Nino, dono do Nino Cucina e Tatu Bola, c...",https://braziljournal.com/alife-nino-dono-do-n...,"saiba mais a sobre Alife Nino, dono do Nino Cu...",2024-04-04 14:41:13.438280,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
53,Trump Media: bolha ou startup?,https://braziljournal.com/trump-media-bolha-ou...,saiba mais a sobre Trump Media: bolha ou startup?,2024-04-04 14:41:13.438687,https://braziljournal.com/wp-content/uploads/2...,Brazil Journal
0,Lula assina MP que promete reduzir conta de lu...,https://www.infomoney.com.br/politica/lula-ass...,Texto abre possibilidade para governo antecipa...,2024-04-09 14:40:42+00:00,https://www.infomoney.com.br/wp-content/upload...,InfoMoney
...,...,...,...,...,...,...
95,Há risco de Fed precisar voltar a elevar juros...,https://valor.globo.com/financas/noticia/2024/...,Michelle Bowman disse que esse não é seu cenár...,2024-04-05 14:40:46.807375,https://s2-valor.glbimg.com/20dVrHtvoFOUhyQbfn...,Valor Econômico
96,Clientes do C6 Bank relatam instabilidade no Pix,https://valor.globo.com/financas/noticia/2024/...,A principal queixa é que transferências realiz...,2024-04-05 14:40:46.807376,https://s2-valor.glbimg.com/ILV8pCrEfd-EynbS04...,Valor Econômico
97,Bolsas da Europa recuam com dados fortes de em...,https://valor.globo.com/financas/noticia/2024/...,"O índice Stoxx 600 caiu 0,88% e anotou seu men...",2024-04-05 14:40:46.807378,https://s2-valor.glbimg.com/7OJTblUtG1oKbqkNyn...,Valor Econômico
98,Fundos de shoppings têm melhor retorno entre F...,https://valor.globo.com/financas/noticia/2024/...,"Segundo o relatório, em 12 meses até março, o ...",2024-04-05 14:40:46.807380,https://s2-valor.glbimg.com/6efJvpNGUZM1ko2Wnm...,Valor Econômico


### Análise de Sentimento das notícias

In [9]:
from transformers import AutoTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd

class SentimentClassifier:
    def __init__(self, model_name="lucas-leme/FinBERT-PT-BR"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name)
        self.pred_mapper = {
            0: "positivo",
            1: "negativo",
            2: "neutro"
        }

    def classify_sentiments(self, df, text_column):
        results = []
        for text in df[text_column]:
            tokens = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            outputs = self.model(**tokens)
            probs = np.exp(outputs.logits.cpu().detach().numpy()) / np.exp(outputs.logits.cpu().detach().numpy()).sum(axis=1, keepdims=True)
            preds = dict(zip(self.pred_mapper.values(), probs[0]))
            results.append(preds)

        # Criar DataFrame com as colunas de probabilidades
        results_df = pd.DataFrame(results)

        # Adicionar as colunas de probabilidades ao DataFrame original usando join
        df_with_probs = df.join(results_df)

        return df_with_probs

# Exemplo de uso
# Supondo que 'df' é o seu DataFrame e 'text_column' é o nome da coluna que contém o texto das notícias
classifier = SentimentClassifier()
df = classifier.classify_sentiments(df, 'titulo')
df

tokenizer_config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

,titulo,url,descricao,data_publicacao,img,fonte,positivo,negativo,neutro
0,Lula assina MP que promete reduzir conta de lu...,https://www.infomoney.com.br/politica/lula-ass...,Texto abre possibilidade para governo antecipa...,2024-04-09 14:40:42+00:00,https://www.infomoney.com.br/wp-content/upload...,InfoMoney,0.199675,0.112629,0.687696
0,Nubank vai desativar app NuInvest e terá plata...,https://valor.globo.com/financas/noticia/2024/...,Os clientes serão migrados para o app do banco...,2024-04-09 14:34:46.807172,https://s2-valor.glbimg.com/gDJUSTtA9mMoRHoZq-...,Valor Econômico,0.199675,0.112629,0.687696
1,"Sabesp (SBSP3) vai reajustar as tarifas em 6,4...",https://www.infomoney.com.br/mercados/sabesp-s...,Reajuste passa a vigorar em maio e irá vigorar...,2024-04-09 14:35:38+00:00,https://www.infomoney.com.br/wp-content/upload...,InfoMoney,0.035194,0.903352,0.061454
1,Ibovespa sobe e dólar cai com commodities e in...,https://valor.globo.com/financas/ao-vivo/2023/...,Investidores aguardam dados de inflação para c...,2024-04-09 14:34:46.807188,https://s2-valor.glbimg.com/A_M_5WNEpu6bfMQ8tf...,Valor Econômico,0.035194,0.903352,0.061454
2,Dividendos da Petrobras (PETR4): acordo sobre ...,https://www.infomoney.com.br/mercados/dividend...,"Governo pode remunerar a mais os acionistas, c...",2024-04-09 14:28:41+00:00,https://www.infomoney.com.br/wp-content/upload...,InfoMoney,0.571538,0.075128,0.353333
...,...,...,...,...,...,...,...,...,...
95,Há risco de Fed precisar voltar a elevar juros...,https://valor.globo.com/financas/noticia/2024/...,Michelle Bowman disse que esse não é seu cenár...,2024-04-05 14:40:46.807375,https://s2-valor.glbimg.com/20dVrHtvoFOUhyQbfn...,Valor Econômico,0.908635,0.032246,0.059119
96,Clientes do C6 Bank relatam instabilidade no Pix,https://valor.globo.com/financas/noticia/2024/...,A principal queixa é que transferências realiz...,2024-04-05 14:40:46.807376,https://s2-valor.glbimg.com/ILV8pCrEfd-EynbS04...,Valor Econômico,0.094956,0.854734,0.050310
97,Bolsas da Europa recuam com dados fortes de em...,https://valor.globo.com/financas/noticia/2024/...,"O índice Stoxx 600 caiu 0,88% e anotou seu men...",2024-04-05 14:40:46.807378,https://s2-valor.glbimg.com/7OJTblUtG1oKbqkNyn...,Valor Econômico,0.490551,0.231738,0.277712
98,Fundos de shoppings têm melhor retorno entre F...,https://valor.globo.com/financas/noticia/2024/...,"Segundo o relatório, em 12 meses até março, o ...",2024-04-05 14:40:46.807380,https://s2-valor.glbimg.com/6efJvpNGUZM1ko2Wnm...,Valor Econômico,0.468207,0.429195,0.102598


### Classificar a empresa

In [21]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Supondo que 'df_noticias' seja o seu dataframe contendo as notícias e os textos das notícias estejam na coluna 'titulo'
# Supondo que 'df_empresas' seja o seu dataframe contendo os nomes das empresas, os IDs das empresas e os nomes estejam nas colunas 'nome' e 'ID' respectivamente
df_noticias = df['titulo']
df_empresas = pd.DataFrame([
    {"nome": "Dexco", "ID": "123"},
    {"nome": "Petrobras", "ID": "124"},
    {"nome": "Tenda", "ID": "125"},
    {"nome": "Vale", "ID": "126"},
    {"nome": "Nubank", "ID": "127"}
])

# Criando um vetorizador TF-IDF para converter o texto das notícias em vetores numéricos
vectorizer = TfidfVectorizer()

# Convertendo o texto das notícias em vetores TF-IDF
tfidf_matrix_noticias = vectorizer.fit_transform(df_noticias)

# Convertendo os nomes das empresas em vetores TF-IDF
nomes_empresas = [empresa['nome'] for empresa in df_empresas.to_dict(orient='records')]
tfidf_matrix_empresas = vectorizer.transform(nomes_empresas)

# Calculando a similaridade de cosseno entre os vetores TF-IDF das notícias e das empresas
similaridade = cosine_similarity(tfidf_matrix_noticias, tfidf_matrix_empresas)

# Definindo um limite de similaridade para considerar uma menção como uma citação de empresa
limite_similaridade = 0.20

# Lista para armazenar os índices das notícias que mencionam empresas
indices_noticias_com_citacoes = []

# Lista para armazenar as empresas mencionadas em cada notícia
empresas_mencionadas_por_noticia = []

# Procurando por notícias que mencionam empresas
for i in range(len(similaridade)):
    empresas_mencionadas = []
    for j in range(len(similaridade[i])):
        if similaridade[i,j] > limite_similaridade:
            empresas_mencionadas.append(df_empresas.iloc[j]['nome'])
    if empresas_mencionadas:
        indices_noticias_com_citacoes.append(i)
        empresas_mencionadas_por_noticia.append(empresas_mencionadas)

# Criando uma lista de dicionários para armazenar os resultados
resultado = []

# Preenchendo a lista com os dados
for empresas, indice in zip(empresas_mencionadas_por_noticia, indices_noticias_com_citacoes):
    for empresa in empresas:
        id_empresa = df_empresas[df_empresas['nome'] == empresa]['ID'].values[0]
        if re.match(r'\b[A-Z][a-z]*\b', empresa):  # Verifica se a primeira letra do nome da empresa é maiúscula
            resultado.append({'Empresa': empresa, 'ID': id_empresa, 'Notícia': df_noticias.iloc[indice]})

# Convertendo a lista de dicionários em um DataFrame
df_resultado = pd.DataFrame(resultado)

# Exibindo o DataFrame resultado
print(df_resultado)


     Empresa   ID                                            Notícia
0     Nubank  127  Nubank vai desativar app NuInvest e terá plata...
1  Petrobras  124  Dividendos da Petrobras (PETR4): acordo sobre ...
2       Vale  126  Ibovespa encerra em alta e se aproxima dos 129...
3     Nubank  127  Nubank obtém financiamento de US$ 150 milhões ...
4     Nubank  127  Nubank: Ações sobem 167% em um ano e banco dig...
5       Vale  126  Ibovespa avança mais de 1% com forte alta de V...
6     Nubank  127  Nubank lança conta global para clientes de alt...
7  Petrobras  124  Manhã no mercado: Petrobras e juros americanos...
